# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [48]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
#Read in weatherpy dataframe
wetdf = pd.read_csv("../output_data/weather.csv")
wetdf.head(10)


,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Windspeed,Date
0,egvekinot,RU,66.3167,-179.1667,47.26,100,92,2.62,1627164681
1,albany,AU,-35.0169,117.8917,54.03,75,87,12.66,1627164681
2,plattsburgh,US,44.6995,-73.4529,81.36,1,46,3.00,1627164681
3,boone,US,39.0334,-92.3335,95.79,6,59,1.99,1627164681
4,prince rupert,CA,54.3161,-130.3201,60.46,81,78,2.48,1627164682
5,hermanus,ZA,-34.4187,19.2345,50.90,14,56,5.68,1627164682
6,saint-philippe,RE,-21.3585,55.7679,67.59,37,79,4.97,1627164682
7,saint george,BM,32.3817,-64.6781,78.58,90,80,13.80,1627164682
8,rikitea,PF,-23.1203,-134.9692,69.26,0,54,1.43,1627164683
9,bethel,US,41.3712,-73.4140,80.46,20,60,8.05,1627164683


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [55]:
# grab maps
gmaps.configure(api_key=g_key)
# store lats/longs and humidity
Locations = wetdf[["Lat", "Lng"]]
hum = wetdf["Humidity"].astype(float)

In [66]:
# Create heatmap
ax = gmaps.figure(center = (46.0, -5.0), zoom_level=2)
max_intensity = np.max(hum)
# Create heat layer
heat = gmaps.heatmap_layer(Locations, weights = hum, max_intensity=100, point_radius=3, dissipating=False)
# Add heat layer and display map
ax.add_layer(heat)
ax

Figure(layout=FigureLayout(height='420px'))

In [57]:
max_intensity

100.0

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [58]:
# Narrow down dataframe to cities w/ windspeed under 10 mph, no clouds, temp between 60-70 degrees F --> display
narrowdf = wetdf.loc[(wetdf["Windspeed"] <= 10) & (wetdf["Cloudiness"] == 0) & \
                     (wetdf["Max Temp"] >= 60) & (wetdf["Max Temp"] <= 70)].dropna()
narrowdf.head(10)

,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Windspeed,Date
8,rikitea,PF,-23.1203,-134.9692,69.26,0,54,1.43,1627164683
21,dingle,IE,52.1408,-10.2689,64.29,0,89,0.96,1627164686
62,villa carlos paz,AR,-31.4241,-64.4978,63.90,0,52,7.00,1627164772
169,belyy yar,RU,53.6039,91.3903,67.89,0,94,2.24,1627164963
253,namibe,AO,-15.1961,12.1522,65.50,0,84,5.97,1627164364
323,bor,TR,37.8906,34.5589,62.33,0,69,7.09,1627165091
367,paracatu,BR,-17.2222,-46.8747,65.05,0,41,3.13,1627165133
417,vigrestad,NO,58.5667,5.7000,61.97,0,88,8.08,1627165175
450,mahajanga,MG,-15.7167,46.3167,66.22,0,94,3.44,1627165214
505,ballina,AU,-28.8667,153.5667,60.98,0,51,9.44,1627165259


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [59]:
# Create hotel_df
hotel_df = narrowdf.loc[:,["City","Country", "Lat", "Lng"]]
# add a "Hotel Name" column
hotel_df["Hotel Name"] = ""
# Display the dataframe
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
8,rikitea,PF,-23.1203,-134.9692,
21,dingle,IE,52.1408,-10.2689,
62,villa carlos paz,AR,-31.4241,-64.4978,
169,belyy yar,RU,53.6039,91.3903,
253,namibe,AO,-15.1961,12.1522,
323,bor,TR,37.8906,34.5589,
367,paracatu,BR,-17.2222,-46.8747,
417,vigrestad,NO,58.5667,5.7000,
450,mahajanga,MG,-15.7167,46.3167,
505,ballina,AU,-28.8667,153.5667,


In [60]:
# set base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#add parameters
params = {"type" : "hotel", "keyword" : "hotel", "radius" : 5000, "key" : g_key}

In [61]:
#loop through hotel dataframe
for index, row in hotel_df.iterrows():
    # get lat long, add parameters
    lng = row["Lng"]
    lat = row["Lat"]
    name = row["City"]
    params["location"] = f"{lat},{lng}"
    # Make request
    print(f"Beginning retrieval for Index {index}: {name}.")
    response = requests.get(base_url, params=params).json()
    result = response['results']
    # save results to hotel dataframe
    try:
        print(f"The closest hotel in {name} is {result[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = result[0]['name']
    #Except if ther is no hotel, skip
    except (KeyError, IndexError):
        print("No result, skipping.")
    #Wait a sec to avoid ssl
    time.sleep(1)
print('''
-----------------------------
Data Retrieval Complete
-----------------------------''')

Beginning retrieval for Index 8: rikitea.
The closest hotel in rikitea is People ThankYou.
Beginning retrieval for Index 21: dingle.
The closest hotel in dingle is Dingle Skellig Hotel.
Beginning retrieval for Index 62: villa carlos paz.
The closest hotel in villa carlos paz is Pinares Panorama Suites & Spa.
Beginning retrieval for Index 169: belyy yar.
No result, skipping.
Beginning retrieval for Index 253: namibe.
The closest hotel in namibe is Hotel Chik Chik Namibe.
Beginning retrieval for Index 323: bor.
The closest hotel in bor is Saray Otel.
Beginning retrieval for Index 367: paracatu.
The closest hotel in paracatu is Hotel Eldorado.
Beginning retrieval for Index 417: vigrestad.
The closest hotel in vigrestad is Ogna Camping AS.
Beginning retrieval for Index 450: mahajanga.
The closest hotel in mahajanga is Baobab Tree Hotel & Spa.
Beginning retrieval for Index 505: ballina.
The closest hotel in ballina is Ramada Hotel & Suites by Wyndham Ballina Byron.

------------------------

In [63]:
# display changes
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
8,rikitea,PF,-23.1203,-134.9692,People ThankYou
21,dingle,IE,52.1408,-10.2689,Dingle Skellig Hotel
62,villa carlos paz,AR,-31.4241,-64.4978,Pinares Panorama Suites & Spa
169,belyy yar,RU,53.6039,91.3903,
253,namibe,AO,-15.1961,12.1522,Hotel Chik Chik Namibe
323,bor,TR,37.8906,34.5589,Saray Otel
367,paracatu,BR,-17.2222,-46.8747,Hotel Eldorado
417,vigrestad,NO,58.5667,5.7000,Ogna Camping AS
450,mahajanga,MG,-15.7167,46.3167,Baobab Tree Hotel & Spa
505,ballina,AU,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron


In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
ax.add_layer(markers)

# Display figure
ax

Figure(layout=FigureLayout(height='420px'))